<h1>Segmenting and Clustering Neighborhoods in Toronto (Part 2)</h1>

In [18]:
#Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd

#Extract the table from the website
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
res = requests.get(url).text
soup = BeautifulSoup(res,'lxml')
table = soup.find('table', class_='wikitable')

#Extract the elements from the table
postcode = []
borough = []
neighborhood = []

for row in table.findAll("tr"):
    cells = row.findAll("td")
    if len(cells) == 3:
        postcode.append(cells[0].find(text=True).rstrip('\n'))
        borough.append(cells[1].find(text=True).rstrip('\n'))
        neighborhood.append(cells[2].find(text=True).rstrip('\n'))
        
#Write the elements in a dataframe
df = pd.DataFrame()
df['Postal Code'] = postcode
df['Borough'] = borough
df['Neighborhood'] = neighborhood

#Remove rows where Borough = Not Assigned
df = df[df["Borough"] != "Not assigned"]

#Group neighbordhoods with same postcode
for i in range(1,df.shape[0]):
    if (df.iloc[i,0] == df.iloc[i-1,0])==True:
        df.iloc[i-1,0] = 'Not assigned'
        df.iloc[i,2] = df.iloc[i-1,2] + ", " + df.iloc[i,2]
           
df = df[df["Postal Code"] != "Not assigned"]

#Assign Borough name to Neighborhood if Neighborhood = Not assigned
for i in range(1,df.shape[0]):
    if (df.iloc[i,2] == "Not assigned")==True:
        df.iloc[i,2] = df.iloc[i,1]
        
df = df.sort_values(by = ["Postal Code"])

df = df.reset_index(drop = True)

df.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [11]:
#Get geographical coordinates from url
geo_coordinates = pd.read_csv('http://cocl.us/Geospatial_data')

geo_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
#Merge the data frames
df = pd.merge(df, geo_coordinates, how='inner', on='Postal Code')

df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [20]:
df.shape

(103, 5)